### Energy Resolution

The goal of this notebook is to compare the achieveable energy resoultion of the optimal filter (gen2) with that of the most successful machine learning model, using the same resonator, readout, and noise parameters. The plan is to have the optimal filter template and filter generated in the same way that would be done during a MEC run, using a dataset consisting of pulses of the same height. The machine learning model will have been trained on a variety of pulses within the same range. The largest pulse height used to train the machine learning model will be used to generate the template/filter in the optimal filter code. The results will then be compared on the single pulse height. The process will be repeated, generating the optimal filter template/filter with one pulse but the input phase timestream will have varied pulse heights using the same range that was used when training the model.

#### Single Pulse Height

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
import os
import random
import torch

from torch.utils.data import TensorDataset
from mkidreadoutanalysis.quasiparticletimestream import QuasiparticleTimeStream
from mkidreadoutanalysis.resonator import Resonator, RFElectronics, ReadoutPhotonResonator, FrequencyGrid, LineNoise
from mkidreadoutanalysis.optimal_filters.make_filters import Calculator
from mkidreadoutanalysis.mkidnoiseanalysis import apply_lowpass_filter, compute_r
from mkidcore.config import ConfigThing
from mlcore.dataset import load_training_data

from mlcore.models import BranchedConvReg
from mlcore.training import make_predictions
from mlcore.dataset import load_training_data, stream_to_arrival
from mlcore.eval import plot_stream_data

First, the calibration data to be used for generating the optimal filter template/filters needs to be imported.
This data is independent of the actual signal data but will also be used as input to the machine learning model so that
the final performance can be compared to the optimal filter. 

In [ ]:
# Define common parameters
EDGE_PAD = 100
NOISE_SCALE = 15
NUM_SAMPLES = 30000
WINDOW = 1000
MAG = 1.000
FS = 1e6
NOISE_ON = True

In [ ]:
# Define data storage parent location
data_parent_dir = os.environ['ML_DATA_DIR']
data_dir = Path(data_parent_dir + '/pulses/test/single_pulse/variable_qp_density/normalized_iq')
p = Path(data_dir, f'vp_single_num{NUM_SAMPLES}_window{WINDOW}_mag{int(MAG * 1000)}_noisescale30_pad{EDGE_PAD}.npz')

# Import the photon arrival data from the stored calibration data
photon_arrs = load_training_data(p, labels=('photon_arrivals',))[0]

# Flatten the array for use with QPT timestream
photon_arrs = photon_arrs.flatten()

In [ ]:
# Create the resonator/noise/readout objects to be used to create the phase response timestream

RES = Resonator(f0=4.0012e9, qi=200000, qc=15000, xa=1e-9, a=0, tls_scale=1e2)
FREQ_GRID = FrequencyGrid(fc=RES.f0, points=1000, span=500e6)
LINE_NOISE = LineNoise(freqs=[60, 50e3, 100e3, 250e3, -300e3, 300e3, 500e3],
                        #amplitudes=[0, 0, 0, 0, 0, 0, 0],
                        #amplitudes=[0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.0001],
                        amplitudes=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.005],
                        #amplitudes=[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01],
                        phases=[0, 0.5, 0,1.3,0.5, 0.2, 2.4],
                        n_samples=100,
                        fs=FS)
RF = RFElectronics(gain=(3.0, 0, 0),
                    phase_delay=0,
                    white_noise_scale=NOISE_SCALE,
                    line_noise=LINE_NOISE,
                    cable_delay=50e-9)

Now that the phase response timestream has been created based on the photon arrivals in the calibration data, the accompanying optimal filter can be built

In [ ]:
# Import a ConfigThing object to store optimal filter nerd knob values
cfg_thing = ConfigThing()

# Now define/store the values in the ConfigThing object
cfg_thing.registerfromkvlist(
    (
        ('dt', 1/FS), # Sampling interval (in secs)
        ('fit', True), # This flag instructs the code to fit or not to fit the created template of photon pulses. The template is made by averaging all the pulses in the passed in phase stream after median subtraction. (by default)
        ('summary_plot', True), # The summary plot gives information about the template creation, filtering and the fit to the template (if enabled)
        ('pulses.unwrap', False), # Phase unwrapping refers to recovering the true phase of a signal after it has been "wrapped" into an arbitrary range (E.g. from -pi to pi). Discontinuities in the wrapped stream typically indicate the phase was wrapped. 
        ('pulses.fallback_template', 'default'), # This tells the code which fallback template to use in case making a "good" template couldn't be made from the data (typically due to not enough pulses in the passed in stream.)
        ('pulses.ntemplate', 1000), # Used in the pulse averaging function. Essentially the length of the pulse template.
        ('pulses.offset', 20), # Offset from the start of the template "window" where the pulse will start.
        ('pulses.threshold', 8), # Only pulses greater than this value multiplied by std dev. of all the filtered pulses will be included in the output.
        ('pulses.separation', 50), # A pulse arriving in a time window shorter than this value (in us) with respect to the previous pulse will be discarded.
        ('pulses.min_pulses', 10000), # Number of pulses needed to make a "good" template.
        ('noise.nwindow', 1000), # The size of the overlapping segment used to create the PSD of the phase stream using Welch's method. This is similar to the window size in the STFT when creating Spectrograms (it's called Periodogram).
        ('noise.isolation', 100), # When making the noise spectrum for the data using Welch's method, having pulses too close to each other can skew results. This parameter helps determine how close is too close.
        ('noise.max_windows', 2000), # maximum number of windows of length nwindow needed when creating the noise spectrum of the phase stream using Welch's method.
        ('noise.max_noise', 5000), # cant seem to find this value anywhere in the Calculator class
        ('template.percent', 80), # Pulses that lie outside the middle "percent" percentiles are discarded when creating the pulse template. Higher number means more fringe pulses are used when making the template.
        ('template.cutoff', 0.1), # The filter response of the chosen filter is 0 for frequencies above this value (units in 1/dt)
        ('template.min_tau', 5), # Tau seems to be variable that parameterizes the integral of the normalized template. This parameter describes the minimum value that tau can have to signify a "good" template.
        ('template.max_tau', 500), # In the context of the previous parameter, this is the max value of tau to consider a template a "good" template
        ('template.fit', 'triple_exponential'), # If fitting the template, this is the fitting function to use. These are defined in the templates.py file.
        ('filter.filter_type', 'wiener'), # The type of filter to use for the optimal filter. These are defined in the filters.py file.
        ('filter.nfilter', 100), # The number of taps to use in the chosen filter. 
                                # Jenny Note: For messing around this should be closer to 1000 and ntemplate should be increased to be 5-10x nfilter
                                # Jenny Note: Need to make sure filter is periodic and this gets hard when the filter is short
        ('filter.normalize', True) # If true, normalizes the filter to a unit response.
    ),
namespace='' # Not relevant for the optimal filter code.
)

# Before sending the phase response time stream to the optimal filter step, it needs to be low-pass filtered.
# The following filter coefficients are pulled from the example notebook in the mkidreadoutanalysis package.
# Current 8-Tap Equirippple Lowpass Exported from MATLAB
coe = np.array([-0.08066211966627938,
                0.02032901400427789,
                0.21182262325068868,
                0.38968583545138658,
                0.38968583545138658,
                0.21182262325068868,
                0.02032901400427789,
                -0.08066211966627938])

Load the ML model

In [ ]:
# Define saved model path
MODEL_DIR = pathlib.Path().cwd() / 'best_models'
MODEL_FNAME = 'cnn_reg_1692139221.pt'

# Device determination
if torch.cuda.is_available():
  device = torch.device("cuda")

elif torch.backends.mps.is_available():
  device = torch.device('mps')

else:
  device = torch.device("cpu")
print(f'Using device: "{device}"')

# Create model instance and load trained model
model = BranchedConvReg(in_channels=2, h_hidden_units=100, h_hidden_layers=3)
model.load_state_dict(torch.load(MODEL_DIR / MODEL_FNAME, map_location=device))

Loop over all the QP density shift magnitudes used in the training the model and get the optimal filter and ml results for each/

In [ ]:
mags = list(800 / np.arange(800, 1400, 100))

# Define empty containers to hold results from the loop
ml_phases_l = []
ofc_phases_l = []

for mag in mags:

    # Generate qp timestream
    print(f'Generating QPT for mag: {mag:.2f}...')
    qptimestream = QuasiparticleTimeStream(FS, 30) # Need the qpt timestream to have a length equal to the calibration stream
    qptimestream.photon_arrivals = photon_arrs # Manually setting the photon arrivals as opposed to having the object generate its own
    qptimestream.gen_quasiparticle_pulse(magnitude=mag)
    _ = qptimestream.populate_photons()
    print(f'Finished generating QPT for mag: {mag:.2f}')

    # Create readout object with updated qp timestream
    readout = ReadoutPhotonResonator(RES, qptimestream, FREQ_GRID, RF, noise_on=True)
    
    # Generate phase response time stream.
    print(f'Generating phase response for mag: {mag:.2f}...')
    phase_response, _ = readout.basic_coordinate_transformation()
    print(f'Finished generating phase response for mag: {mag:.2f}')

    # Generate optimal filter object and plot the summary
    low_pass_resp = apply_lowpass_filter(coe, phase_response)
    optimal_filter = Calculator(low_pass_resp, config=cfg_thing)
    optimal_filter.calculate()
    print(f'Summary Plot for mag: {mag:.2f}')
    optimal_filter.plot()
    plt.show()

    # Generate I/Q streams from the QPT object for use in model
    print(f'Generating I/Q timestreams for mag: {mag:.2f}...')
    I = readout.normalized_iq.real
    Q = readout.normalized_iq.imag
    print(f'Finished generating I/Q timestreams for mag: {mag:.2f}')

    # Reshape the arrays such that they can be used in the ML data transformation
    # functions.
    I = I.reshape(30000, 1000)
    Q = Q.reshape(30000, 1000)

    # Transform data for model
    i = np.expand_dims(I, axis=1)
    q = np.expand_dims(Q, axis=1)

    # Get pulse heights and photon arrival values
    target_arrs = stream_to_arrival(photon_arrs.reshape(30000, 1000))
    target_pulse = np.min(phase_response.reshape(30000, 1000), axis=1, keepdims=True)

    # Now we want to convert the loaded data to tensors.
    # Shape for targets is NUM_SAMPLES x 1 x 2j
    # Shape for inputs is NUM_SAMPLES x 2 x WINDOW_SIZE
    X = torch.Tensor(np.hstack((i, q)))
    y = torch.Tensor(np.stack((target_arrs, target_pulse), axis=2))

    # From the newly created tensors, create testing and training datasets
    dataset = TensorDataset(X, y)

    # Make model predictions
    samples = []
    labels = []

    for sample, label in random.sample(list(dataset), k=len(dataset)): # random.sample randomly samples k elements from the given population without replacement; returns list of samples.
        samples.append(sample)
        labels.append(label)

    print(f'Generating model predictions for mag: {mag:.2f}...')
    preds = make_predictions(model, [x.unsqueeze(dim=0) for x in samples], device=device) # returns a list
    print(f'Finished generating model predictions for mag: {mag:.2f}...')
    
    # Apply filtering to the optimal filter stream and get filtered phase responses
    _ = optimal_filter.apply_filter()
    ofc_phase_preds, _ = optimal_filter.compute_responses(threshold=cfg_thing.get('pulses.threshold'))
    
    # Now randomly pick number of samples from the ml predictions based on the number of
    # pulses in the ofc phase preds
    sampled_ml_preds = [pred for pred in random.sample(preds, k=ofc_phase_preds.size)]
    
    # Create np array of the sampled ml preds
    model_phase_preds = np.array([pred[1] for pred in sampled_ml_preds])

    # Add results to containers
    print(f'Collecting results...')
    ml_phases_l.append(model_phase_preds)
    ofc_phases_l.append(ofc_phase_preds)

With all the phase response results in the appropriate container, can now plot as necessary. Lets split the plots based on source (ML or optimal filter).

In [ ]:
### ML Results
n_bins = 200


plt.figure(figsize=(12,7))
# Loop through all the results and add the hist of the result to the figure
for phase, mag in zip(ml_phases_l, mags):
    r = compute_r(phase)
    counts, bins = np.histogram(phase, range=(-2.2, -1), bins=n_bins)
    plt.stairs(counts, bins, label=f'QP Shift Mag: {mag:.3f}, "R":{r:.1f}')

plt.title(f'Predicted Pulse Heights (ML), White Noise Scale: {NOISE_SCALE}')
plt.xlabel('Pulse Height')
plt.ylabel('Counts')
plt.legend()
plt.show()

In [ ]:
### Optimal filter Results
n_bins = 200


plt.figure(figsize=(12,7))
# Loop through all the results and add the hist of the result to the figure
for phase, mag in zip(ofc_phases_l, mags):
    r = compute_r(phase)
    counts, bins = np.histogram(phase, range=(-2.3, -1), bins=n_bins)
    plt.stairs(counts, bins, label=f'QP Shift Mag: {mag:.3f}, R:{r:.1f}')

plt.title(f'Predicted Pulse Heights (Optimal Filter), White Noise Scale: {NOISE_SCALE}')
plt.xlabel('Pulse Height')
plt.ylabel('Counts')
plt.legend()
plt.show()